## Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

**Goal:** Create `traders.py` that will manage a single trader on our trading floor.

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


### MCP Params for Trader

In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### MCP Params for Researcher

In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Instantiate MCP servers for each of the above MCP parameters

In [5]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Creating a Researcher Agent to do market research

In [6]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [7]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [ ]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))

Here is the latest news summary on Amazon as of now:

1. Amazon is actively investing in AI and innovation. Real-time analytics with custom AI models are enhancing experiences in sports broadcasting like Thursday Night Football, NASCAR, and NBA. Amazon Alexa+ has introduced conversational AI features to revolutionize music discovery and enjoyment.

2. The Cyber Monday event kicked off on November 29, 2025, highlighting significant promotions and deals for Prime members.

3. Amazon is focusing on supporting small businesses and improving the shopping experience with AI-powered tools.

4. On the workforce front, Amazon provides healthcare and well-being resources starting day one of employment. They also offer educational benefits and training programs such as the Career Choice program to help employees develop new skills.

5. In terms of investment, Amazon plans a large scale investment of up to $50 billion to expand its AI and supercomputing capabilities, especially targeting US government agencies and AWS infrastructure.

6. Amazon expects significant growth, aiming to double sales by 2033, partly by increasing automation and robotics to handle a large workforce demand.

If you want, I can provide more detailed information on specific topics such as Amazon's AI investments, Cyber Monday deals, or workforce initiatives. Would you like that?

[non-fatal] Tracing client error 400: {
  "error": {
    "message": "Invalid type for 'data[0].span_data.result': expected an array of strings, but got null instead.",
    "type": "invalid_request_error",
    "param": "data[0].span_data.result",
    "code": "invalid_type"
  }
}


Check the traces: https://platform.openai.com/traces

### Creating traders

In [9]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-11-30 01:55:22", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

In [10]:
agent_name = "Ed"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [11]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4.1-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

I researched the current market opportunities and notable financial news for day trading. Based on the momentum and news:

1. I bought 20 shares of Inspire Medical Systems (ISPC) following its strong upgrade and price target increase.
2. I also bought 50 shares of Walmart (WMT) on the strength of its recent earnings and positive outlook.

These trades fit the strategy of capitalizing on news-driven momentum and volatility. My current portfolio holdings now include ISPC and WMT, with a cash balance of around $4,455 remaining for further trading opportunities. I will continue to monitor the news and market conditions for additional trades.

Check the traces: https://platform.openai.com/traces

In [12]:
# results of the trading

await read_accounts_resource(agent_name)

'{"name": "ed", "balance": 4455.030195999999, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"ISPC": 20, "WMT": 50}, "transactions": [{"symbol": "ISPC", "quantity": 20, "price": 0.4209402, "timestamp": "2025-11-30 01:55:48", "rationale": "Buying Inspire Medical Systems due to strong upgrade, price target increase, and momentum for short-term gains."}, {"symbol": "WMT", "quantity": 50, "price": 110.73102, "timestamp": "2025-11-30 01:55:48", "rationale": "Buying Walmart on strong earnings and outlook as a news-driven trade opportunity."}], "portfolio_value_time_series": [["2025-11-30 01:55:22", 10000.0], ["2025-11-30 01:55:25", 10000.0], ["2025-11-30 01:55:48", 9999.983196], ["2025-11-30 01:55:48", 9988.932195999998], ["2025-11-30 01:59:24", 9988.932195999998]], "total_portfolio_value": 9988.932195999998, "total_profit_loss": -11.06780400000116}'

In cases where we are going to use multiple servers, use the code below:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

In [20]:
from traders import Trader

trader = Trader("Ed")

In [21]:
await trader.run()

In [22]:
await read_accounts_resource("Ed")

'{"name": "ed", "balance": 1314.661995999999, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"ISPC": 20, "WMT": 50, "DIS": 30}, "transactions": [{"symbol": "ISPC", "quantity": 20, "price": 0.4209402, "timestamp": "2025-11-30 01:55:48", "rationale": "Buying Inspire Medical Systems due to strong upgrade, price target increase, and momentum for short-term gains."}, {"symbol": "WMT", "quantity": 50, "price": 110.73102, "timestamp": "2025-11-30 01:55:48", "rationale": "Buying Walmart on strong earnings and outlook as a news-driven trade opportunity."}, {"symbol": "DIS", "quantity": 30, "price": 104.67894, "timestamp": "2025-11-30 13:20:04", "rationale": "Investing in Disney due to strong market recovery signs and significant earnings potential."}], "portfolio_value_time_series": [["2025-11-30 01:55:22", 10000.0], ["2025-11-30 01:55:25", 10000.0], ["2025-11-30 01:55:48", 9999.983196], ["2025-11-30 01:55:48", 9988.

**Total number of tools used in this project**

In [23]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 16 tools
